In [11]:
from pandas.testing import assert_frame_equal
from visual_behavior.change_detection.trials import summarize
import datetime
import pytz

import pytest
import py
import os
import tempfile
import datetime
from dateutil.tz import tzutc
import numpy as np
import pandas as pd
import pytz
from six import PY3
from six.moves import cPickle as pickle
import uuid



from visual_behavior.pizza import we_can_unpizza_that  # this is terrible but hopefully will be an external dependency very soon
from visual_behavior.uuid_utils import make_deterministic_session_uuid


/home/dougo/Code/visual_behavior_analysis/visual_behavior/pizza/io.py:20: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  pd.DataFrame, pd.Series, pd.Panel,


In [15]:
def _epoch_summary():
    summary = pd.read_csv(
        os.path.join(TESTING_RES_DIR, 'epoch_level_summary.csv'),
        index_col=0,
    )
    summary['startdatetime'] = datetime.datetime(2017, 7, 19, 10, 35, 8, 369000, tzinfo=pytz.utc)
    summary['startdatetime'] = pd.to_datetime(summary['startdatetime'])
    summary['behavior_session_uuid'] = summary['behavior_session_uuid'].map(uuid.UUID)
    return summary

In [37]:
def _mock_trials_fixture():

    n_tr = 500
    np.random.seed(42)
    change = np.random.random(n_tr) > 0.8
    incorrect = np.random.random(n_tr) > 0.8
    detect = change.copy()
    detect[incorrect] = ~detect[incorrect]
    
    change_times = np.sort(np.random.rand(n_tr)) * 3600
    start_times = change_times 

    trials = pd.DataFrame({
        'change': change,
        'detect': detect,
    },)
    trials['trial_type'] = trials['change'].map(lambda x: ['catch', 'go'][x])
    trials['response'] = trials['detect']
    trials['change_time'] = change_times
    trials['reward_lick_latency'] = 0.1
    trials['reward_lick_count'] = 10
    trials['auto_rewarded'] = False
    trials['lick_times'] = [[] for row in trials.iterrows()]
    trials['lick_frames'] = [[] for row in trials.iterrows()]
    trials['starttime'] = start_times
    trials['number_of_rewards'] = 0
    trials['trial_length'] = 8.5
    trials['reward_times'] = trials.apply(lambda r: [r['change_time'] + 0.2] if r['change'] * r['detect'] else [], axis=1)
    trials['reward_volume'] = 0.005 * trials['reward_times'].map(len)
    trials['response_latency'] = trials.apply(lambda r: 0.2 if r['detect'] else np.inf, axis=1)
    trials['blank_duration_range'] = [[0.5, 0.5] for row in trials.iterrows()]

    metadata = {}
    metadata['mouse_id'] = 'M999999'
    metadata['user_id'] = 'johnd'

    metadata['startdatetime'] = datetime.datetime(2017, 7, 19, 10, 35, 8, 369000, tzinfo=pytz.utc)
    metadata['dayofweek'] = metadata['startdatetime'].weekday()
    metadata['startdatetime'] = metadata['startdatetime']

    metadata['behavior_session_uuid'] = make_deterministic_session_uuid(
        metadata['mouse_id'],
        metadata['startdatetime'].isoformat(),
    )
    metadata['stage'] = 'test'
    metadata['stimulus'] = 'natural_scenes'
    metadata['stimulus_distribution'] = 'exponential'

    for k, v in metadata.items():
        trials[k] = v
    return trials

In [44]:
TESTING_RES_DIR = '/home/dougo/code/visual_behavior_analysis/tests/res'
mock_trials_fixture = _mock_trials_fixture()
epoch_summary = _epoch_summary()

[17.78393136 20.73117779 22.98913818]


In [85]:

mock_trials_fixture = _mock_trials_fixture()
summary = summarize.epoch_level_summary(mock_trials_fixture)
epoch_summary = _epoch_summary()
assert_frame_equal(
    summary,
    epoch_summary,
    check_names=False,
    check_like=True,
    check_dtype=False,
    check_exact=False,
    check_less_precise=2,
)

[17.78393136 20.73117779 22.98913818]


In [66]:
DataFrame.iloc[:, 4] (column name="hit_rate") values are different (83.33333 %)
[left]:  [1.4126380446933737, 2.524188847753093, 2.1385199046623367, 1.419707075433647, 2.314600164442237, 1.8331835028400612]
[right]: [1.345565013, 2.65994147, 2.124602583, 1.69922965, 2.0971077830000002, 1.833183503]

SyntaxError: invalid syntax (<ipython-input-66-a25ce1bde55f>, line 1)

In [84]:
summary[['number_of_go_trials','number_of_catch_trials','flashwise_lick_probability','criterion']]

metrics,number_of_go_trials,number_of_catch_trials,flashwise_lick_probability,criterion
0,17.0,70.0,0.0,-0.222580
1,16.0,59.0,0.0,0.111745
2,13.0,63.0,0.0,-0.356817
3,20.0,57.0,0.0,0.035364
4,18.0,73.0,0.0,-0.063340
5,22.0,72.0,0.0,0.168733


In [23]:
summarize.epoch_level_summary(mock_trials_fixture)

metrics,mouse_id,behavior_session_uuid,startdatetime,epoch,num_contingent_trials,response_bias,discrim_p,flashwise_lick_probability,earned_water,lick_latency_median,fraction_time_aborted,number_of_go_trials,number_of_catch_trials,hit_rate,false_alarm_rate,d_prime,reward_lick_count,reward_lick_latency,"mean(HR,FA)",criterion
0,M999999,bedb16bc-e71b-5644-a5cc-e4719a0e2ae7,2017-07-19 10:35:08.369000+00:00,0.0,500.0,0.326,4.388484e-35,0.0,0.44,0.2,0.0,106.0,394.0,0.830189,0.190355,1.831499,0.0,0.1,0.510272,-0.039162


In [26]:
mock_trials_fixture[['change_time']]

,change_time
0,17.783931
1,20.731178
2,22.989138
3,35.175051
4,41.833944
...,...
495,3581.668162
496,3586.802098
497,3589.648201
498,3592.562798
